#### Visualization of Xenium add-on panel healthy gut data after the labels transer 
- **Developed by:** Anna Maguza
- **Affilation:** Wuerzburg Institute for System Immunology
- **Created date:** 20th March 2024
- **Last modified date:** 31st July 2024

##### Import packages

In [1]:
import anndata as ad
import scanpy as sc
import squidpy as sq
import pandas as pd
from scipy.io import mmread
import matplotlib.pyplot as plt
import numpy as np
import scipy as sci
from scipy.sparse import coo_matrix
import seaborn as sns

#### Set up working environment

In [2]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

-----
anndata     0.10.5.post1
scanpy      1.9.8
-----
PIL                         10.2.0
anyio                       NA
arrow                       1.3.0
asciitree                   NA
asttokens                   NA
attr                        23.2.0
attrs                       23.2.0
babel                       2.14.0
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
brotli                      1.1.0
certifi                     2023.11.17
cffi                        1.16.0
charset_normalizer          3.3.2
cloudpickle                 3.0.0
comm                        0.2.1
cycler                      0.12.1
cython_runtime              NA
dask                        2024.1.1
dask_image                  2023.08.1
datashader                  0.16.0
datatree                    0.0.13
dateutil                    2.8.2
debugpy                     1.8.0
decorator                   5.1.1
defusedxml                  0.7.1
docrep                   

#### Data upload

+ Load h5ad object

In [3]:
adata_xenium = sc.read_h5ad('/../../../gut_project/Processed_data/Gut_data/10X_Xenium_data/10X_Xenium_add-on_without_image.h5ad')

#### Visualise predicted cell types - after cell types + Stem cells labels transfer

+ Upload the object with cell types

In [4]:
adata_full = sc.read_h5ad('/../../../gut_project/Processed_data/Gut_data/10X_Xenium_data/Xenium_add_on_data_scANVI_cell_type_and_stem_cells.h5ad')

In [5]:
adata_full.obs.index.name = 'cell_id'

In [6]:
adata_xenium.obs.index.name = 'cell_id'

In [7]:
del adata_xenium.obs['cell_id']

In [8]:
adata_xenium.obs = adata_xenium.obs.merge(adata_full.obs[['C_scANVI']], on='cell_id', how='left')

In [9]:
adata_xenium.obs.rename(columns={'C_scANVI':'Cell_Type'}, inplace=True)

+ Visualize the cell types with standard color pallet

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(10, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True, img_res_key='lowres',
    shape=None,
    color=[
        "Cell_Type",
    ],
    size=0.1,
    alpha=1
)

In [11]:
adata_xenium.uns['Cell_Type_colors']

['#1f77b4',
 '#ff7f0e',
 '#279e68',
 '#d62728',
 '#aa40fc',
 '#8c564b',
 '#e377c2',
 '#b5bd61',
 '#17becf']

+ Change the color pallet to highlight stem cells

In [ ]:
# Define your custom color palette
custom_colors = [
    "#41729F",  # B cells
    "#A16AE8",  # Endothelial
    "#B6E5D8",  # Epithelial
    "#EC9EC0",  # Mesenchymal + 
    "#FBE5C8",  # Myeloid + 
    "#4D5D53",  # Neuronal + 
    "#870052",  # Plasma cells + 
    "#FF0000FF",  # Stem cells + 
    "#ffde17"   # T cells +
]

# Update the color codes in the AnnData object
adata_xenium.uns['Cell_Type_colors'] = custom_colors

# Now, when you plot using sq.pl.spatial_scatter, it should use your updated custom colors
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True,
    img_res_key='lowres',
    shape=None,
    color="Cell_Type",
    size=0.1,
    alpha=1
)

+ Add stem cells types

In [11]:
adata_stem_cells = sc.read_h5ad('/../../../gut_project/Processed_data/Gut_data/10X_Xenium_data/Stem_cells_scRNA_and_Xenium_add_on_scANVI_7K_cells.h5ad')

In [12]:
adata_stem_cells = adata_stem_cells[adata_stem_cells.obs['Study_name'] == '10X_Xenium_gut_data']

In [13]:
adata_stem_cells = adata_stem_cells[adata_stem_cells.obs.index.isin(adata_xenium.obs.index)]

In [14]:
adata_xenium.obs['Stem_cells'] = 'Unknown'
adata_xenium.obs.loc[adata_stem_cells.obs.index, 'Stem_cells'] = adata_stem_cells.obs['C_scANVI']

In [15]:
adata_xenium.obs['CellType2'] = adata_xenium.obs['Cell_Type'].copy()
adata_xenium.obs['CellType2'] = adata_xenium.obs['CellType2'].cat.add_categories(['MTRNR2L12+ASS1+_SC',
                                                                                  'RPS10+_RPS17+_SC',
                                                                                  'FXYD3+_CKB+_SC'])

adata_xenium.obs.loc[(adata_xenium.obs['Stem_cells'] == 'MTRNR2L12+ASS1+_SC'), 'CellType2'] = 'MTRNR2L12+ASS1+_SC'
adata_xenium.obs.loc[(adata_xenium.obs['Stem_cells'] == 'RPS10+_RPS17+_SC'), 'CellType2'] = 'RPS10+_RPS17+_SC'
adata_xenium.obs.loc[(adata_xenium.obs['Stem_cells'] == 'FXYD3+_CKB+_SC'), 'CellType2'] = 'FXYD3+_CKB+_SC'

In [16]:
adata_xenium.obs['CellType2'] = adata_xenium.obs['CellType2'].cat.remove_unused_categories()

+ Visualize cell types + stem cells populations with the standard color pallet

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(10, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True, img_res_key='lowres',
    shape=None,
    color=[
        "CellType2",
    ],
    size=0.1,
    alpha=1
)

+ Change color pallete

In [ ]:
# Define your custom color palette
custom_colors = [
    "#41729F",  # B cells
    "#A16AE8",  # Endothelial
    "#B6E5D8",  # Epithelial
    "#EC9EC0",  # Mesenchymal + 
    "#FBE5C8",  # Myeloid + 
    "#4D5D53",  # Neuronal + 
    "#870052",  # Plasma cells +  
    "#ffde17",   # T cells +
    "#00B1B0",  # Stem cells MTRN
    "#508FF7",  # Stem cells RPS
    "#FF0000FF"  # Stem cells FXYD
]

# Update the color codes in the AnnData object
adata_xenium.uns['CellType2_colors'] = custom_colors

# Now, when you plot using sq.pl.spatial_scatter, it should use your updated custom colors
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True,
    img_res_key='lowres',
    shape=None,
    color="CellType2",
    size=0.1,
    alpha=1
)

In [ ]:
adata_xenium.write_h5ad('/../../../gut_project/Processed_data/Gut_data/10X_Xenium_data/Xenium_add_on_celltypes_and_stem_cells_types_without_image.h5ad')

+ Visualise only Stem cells

In [ ]:
custom_colors = [
    "#41729F",  # B cells
    "#A16AE8",  # Endothelial
    "#B6E5D8",  # Epithelial
    "#EC9EC0",  # Mesenchymal + 
    "#FBE5C8",  # Myeloid + 
    "#4D5D53",  # Neuronal + 
    "#870052",  # Plasma cells +  
    "#ffde17",   # T cells +
    "#00B1B0",  # Stem cells MTRN
    '#FFDB15',  # Stem cells RPS
    "#FF0000FF"  # Stem cells FXYD
]

# Update the color codes in the AnnData object
adata_xenium.uns['CellType2_colors'] = custom_colors

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(20, 10))

cell_types_of_interest = ['MTRNR2L12+ASS1+_SC', 'RPS10+_RPS17+_SC', 'FXYD3+_CKB+_SC']

stem_cells_mask = adata_xenium.obs['CellType2'].isin(cell_types_of_interest)

In [ ]:
adata_stem_cells = adata_xenium[stem_cells_mask, :]

In [ ]:
sq.pl.spatial_scatter(
    adata_stem_cells,  # Use the filtered AnnData object.
    library_id="spatial",
    img=True,
    shape=None,  # Assuming you want the default shape.
    color="CellType2",  # This will now only show 'Stem cells'.
    size=0.1,
    alpha=1
)

+ Add other cell states

In [18]:
adata_scvi = sc.read_h5ad('/../../../gut_project/Processed_data/Gut_data/10X_Xenium_data/Xenium_add_on_data_scANVI_cell_states.h5ad')

In [19]:
adata_scvi.obs.index.name = 'cell_id'

In [20]:
# merge adata_scvi.obs['C_scANVI'] to adata_xenium
adata_xenium.obs = adata_xenium.obs.merge(adata_scvi.obs[['C_scANVI']], on='cell_id', how='left')

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(20, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True,
    img_res_key='lowres',
    shape=None,
    color="C_scANVI",
    size=0.1,
    alpha=1
)

In [22]:
adata_xenium.obs['C_scANVI'].value_counts()

C_scANVI
Myofibroblasts          41555
Fibroblasts             34286
Colonocyte              33628
IgA plasma cell         18934
Stem cells              17780
Enterocyte              14629
Macrophages             12686
Goblet cells            11963
CD4 T                   10705
B cells                 10679
Glial cells              8114
Pericytes                6723
TA                       6644
DC                       6299
BEST4+ epithelial        5687
arterial capillary       4737
EECs                     3784
Tuft cells               3671
Mature venous EC         3631
Mast cells               2998
Plasma cells             2495
CD8 T                    2428
Monocytes                2330
Mature arterial EC       2312
LEC                      1730
Memory B                 1523
Immune Cycling cells     1213
Naive B                   317
NK                        305
Microfold cell            184
ILCs                      169
gdT                        39
B cells AICDA LRMP         34
T

In [23]:
adata_xenium.obs

,x_centroid,y_centroid,transcript_counts,control_probe_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,nucleus_area,...,log1p_total_counts,pct_counts_in_top_10_genes,pct_counts_in_top_20_genes,pct_counts_in_top_50_genes,pct_counts_in_top_150_genes,n_counts,Cell_Type,Stem_cells,CellType2,C_scANVI
cell_id,,,,,,,,,,,,,,,,,,,,,
aaaagkoa-1,830.147217,836.563416,284,0,0,0,0,284.0,653.365805,23.120001,...,5.652489,58.450704,76.056338,96.830986,100.0,284.0,Mesenchymal,Unknown,Mesenchymal,Fibroblasts
aaablbol-1,735.468018,854.389282,116,0,0,0,0,116.0,651.243461,12.643750,...,4.762174,65.517241,82.758621,100.000000,100.0,116.0,Mesenchymal,Unknown,Mesenchymal,Fibroblasts
aaabogma-1,847.423462,846.633240,170,0,0,0,0,170.0,674.544087,18.152813,...,5.141664,72.941176,92.941176,100.000000,100.0,170.0,Mesenchymal,Unknown,Mesenchymal,Fibroblasts
aaabpdim-1,1543.696045,4223.798340,61,0,0,0,0,61.0,172.406569,11.379375,...,4.127134,75.409836,91.803279,100.000000,100.0,61.0,Mesenchymal,Unknown,Mesenchymal,Myofibroblasts
aaabpfoj-1,1551.099121,4220.062988,105,0,0,0,0,105.0,283.490948,56.806565,...,4.663439,73.333333,87.619048,100.000000,100.0,105.0,Mesenchymal,Unknown,Mesenchymal,Myofibroblasts
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
oikabfkp-1,8284.036133,3061.574951,219,0,0,0,0,219.0,70.308284,38.563439,...,5.393628,56.164384,79.908676,100.000000,100.0,219.0,Epithelial,Unknown,Epithelial,Colonocyte
oikalpjm-1,8267.514648,3067.372314,173,0,0,0,0,173.0,45.743283,11.334219,...,5.159055,54.913295,75.722543,100.000000,100.0,173.0,Epithelial,Unknown,Epithelial,Colonocyte
oikamhom-1,8260.333984,3068.529053,369,0,0,0,0,369.0,104.040004,46.691564,...,5.913503,49.864499,73.170732,95.663957,100.0,369.0,Epithelial,Unknown,Epithelial,Colonocyte


+ add all possible cell states

In [24]:
adata_scvi_2 = sc.read_h5ad('/../../../gut_project/Processed_data/Gut_data/10X_Xenium_data/Xenium_add_on_data_scANVI_cell_states_all.h5ad')

In [25]:
adata_scvi_2.obs.index.name = 'cell_id'

In [26]:
adata_scvi_2.obs.rename(columns={'C_scANVI':'Predicted_cell_states'}, inplace=True)

In [27]:
adata_xenium.obs = adata_xenium.obs.merge(adata_scvi_2.obs[['Predicted_cell_states']], on='cell_id', how='left')

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(20, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True,
    img_res_key='lowres',
    shape=None,
    color="Predicted_cell_states",
    size=0.1,
    alpha=1
)

In [28]:
adata_xenium.write_h5ad('/../../../gut_project/Processed_data/Gut_data/10X_Xenium_data/Xenium_add_on_cell_states.h5ad')

In [ ]:
crosstab = pd.crosstab(adata_xenium.obs['C_scANVI'], adata_xenium.obs['Predicted_cell_states'])

# Create the heatmap
plt.figure(figsize=(20, 20))
sns.heatmap(crosstab, cmap='magma_r', square=True, cbar=False, linewidths=0)

plt.xticks(rotation=90, ha='center')

plt.tight_layout()
plt.show()